## Importing Libraries and Data Preparation

In [37]:
# Import necessary libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import SimpleRNN, Dense, Input

# Function to generate alternating bit sequences
def generate_bit_sequence(length):
    return np.array([i % 2 for i in range(length)], dtype=np.float32).reshape(-1, 1)

# Function to prepare data for training
def prepare_data(sequence, window_size=2):
    X, y = [], []
    for i in range(len(sequence) - window_size):
        X.append(sequence[i:i+window_size])
        y.append(sequence[i+window_size])
    return np.array(X), np.array(y)


## RNN Model Construction and Compilation

In [38]:
# Hyperparameters
sequence_length = 20  # Length of the bit string
window_size = 2  # Size of the input window
batch_size = 1  # Size of the batch
epochs = 100  # Number of training epochs

# Generate the alternating bit sequence
bit_sequence = generate_bit_sequence(sequence_length)

# Prepare the data
X_train, y_train = prepare_data(bit_sequence, window_size)

# Reshape the data for RNN input (samples, time steps, features)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))

inputs = Input(shape=(X_train.shape[1], 1))
rnn_layer = SimpleRNN(units=10, return_sequences=False)(inputs)
outputs = Dense(1, activation='sigmoid')(rnn_layer)
model = Model(inputs=inputs, outputs=outputs)


# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


## Model Training and Testing 

In [39]:
# Train the model
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1)

# Test the model on a new sequence
def test_rnn(input_sequence):
    input_sequence = np.array(input_sequence).reshape((1, len(input_sequence), 1))
    predicted_bit = model.predict(input_sequence)
    return 1 if predicted_bit > 0.5 else 0

# Test cases
test_input_1 = [0, 1]
test_input_2 = [1, 0]

print(f"Input: {test_input_1}, Predicted next bit: {test_rnn(test_input_1)}")
print(f"Input: {test_input_2}, Predicted next bit: {test_rnn(test_input_2)}")


Epoch 1/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 817us/step - accuracy: 1.0000 - loss: 0.4595 
Epoch 2/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 783us/step - accuracy: 1.0000 - loss: 0.4348
Epoch 3/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 695us/step - accuracy: 1.0000 - loss: 0.3817
Epoch 4/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 734us/step - accuracy: 1.0000 - loss: 0.3409
Epoch 5/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 693us/step - accuracy: 1.0000 - loss: 0.3073
Epoch 6/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 659us/step - accuracy: 1.0000 - loss: 0.2735
Epoch 7/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 668us/step - accuracy: 1.0000 - loss: 0.2410
Epoch 8/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 626us/step - accuracy: 1.0000 - loss: 0.2246
Epoch 9/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 678us/step - accuracy: 1.0000 - loss: 0.1939
Epoch 10/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 632us/step - accuracy: 1.0000 - loss: 0.1676
Epoch 11/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 672us/step - accuracy: 1.0000 - loss: 0.1652
Epoch 12/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 

## Visualizing the Hidden state 

In [40]:
# Create a model that returns the hidden state of the RNN layer
hidden_state_model = Model(inputs=model.input, outputs=model.layers[1].output)

# Function to get hidden state and predicted output
def get_hidden_state_and_output(input_sequence):
    input_sequence = np.array(input_sequence).reshape((1, len(input_sequence), 1))
    hidden_state = hidden_state_model.predict(input_sequence)
    output = model.predict(input_sequence)
    return hidden_state, output

# Test cases
test_input_1 = [0, 1]
test_input_2 = [1, 0]

# Get hidden state and output for test sequences
hidden_state_1, output_1 = get_hidden_state_and_output(test_input_1)
hidden_state_2, output_2 = get_hidden_state_and_output(test_input_2)

# Print hidden states and outputs
print(f"Input: {test_input_1}")
print(f"Hidden state: {hidden_state_1}")
print(f"Predicted output: {output_1}\n")

print(f"Input: {test_input_2}")
print(f"Hidden state: {hidden_state_2}")
print(f"Predicted output: {output_2}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Input: [0, 1]
Hidden state: [[ 0.6860001   0.80083627 -0.757319   -0.7251877   0.5985031   0.8486418
   0.8334003  -0.7846172   0.87092274 -0.6069351 ]]
Predicted output: [[0.00260896]]

Input: [1, 0]
Hidden state: [[-0.82056016 -0.8855791   0.7749776   0.8059531  -0.80147576 -0.8346672
  -0.9244355   0.9530324  -0.8773409   0.3532817 ]]
Predicted output: [[0.9981462]]
